In [124]:
import numpy as np
import scipy
from utils.SVD import svd_phaseI, svd_phaseIIA, svd, eigh_by_QR

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Problem 1: SVD by Two-Phase Approach

## Phase-I: Golub-Kahan Bidiagonalization

In [125]:
A = np.array([[1, 0, 1],
              [2, 5**.5, 0],
              [0, 0, 1],
              [0, 0, 1]])
B, Qt, P = svd_phaseI(A)
print(B)


[[-2.23606798e+00  2.04939015e+00 -9.45175510e-17]
 [ 1.11022302e-16  8.39500986e-01 -5.07346289e-01]
 [ 0.00000000e+00 -2.67014343e-18  1.68458833e+00]
 [ 0.00000000e+00 -4.39009789e-18 -3.33066907e-16]]


In [126]:
Qt[:3, :] @ A @ P

array([[-2.23606798e+00,  2.04939015e+00,  9.45175510e-17],
       [-1.11022302e-16,  8.39500986e-01, -5.07346289e-01],
       [ 5.55111512e-17, -2.00911092e-17,  1.68458833e+00]])

Todo: We may not need to form the reflection matrix H explicitly during Golub-Kahan bidiagonalization.

## Phase-II-A

In [127]:
T, Q = eigh_by_QR(B.T@B)
T, Q

(array([9.54913355, 3.11464321, 0.33622324]),
 array([[ 0.70892578, -0.10995498, -0.69665927],
        [-0.70375227, -0.04523752, -0.70900374],
        [ 0.04644335,  0.99290658, -0.10945111]]))

In [128]:
scipy.linalg.eigh(B.T@B)

(array([0.33622324, 3.11464321, 9.54913355]),
 array([[ 0.69665927, -0.10995498,  0.70892578],
        [ 0.70900374, -0.04523752, -0.70375227],
        [ 0.10945111,  0.99290658,  0.04644335]]))

**Test SVD**

In [129]:
%precision 8

'%.8f'

In [130]:
svd(A)

(array([[-0.26444298, -0.60575682, -0.75041894],
        [-0.96312812,  0.2058522 ,  0.17323137],
        [-0.03502958, -0.54345356,  0.45103343],
        [-0.03502958, -0.54345356,  0.45103343]]),
 array([3.09016724, 1.76483518, 0.5798476 ]),
 array([[ 0.70892578,  0.69692666,  0.10824726],
        [-0.10995498,  0.26081728, -0.95910596],
        [-0.69665927,  0.66803262,  0.26153065]]))

In [131]:
scipy.linalg.svd(A, full_matrices=False)

(array([[-0.26444298,  0.60575682,  0.75041894],
        [-0.96312812, -0.2058522 , -0.17323137],
        [-0.03502958,  0.54345356, -0.45103343],
        [-0.03502958,  0.54345356, -0.45103343]]),
 array([3.09016724, 1.76483518, 0.5798476 ]),
 array([[-0.70892578, -0.69692666, -0.10824726],
        [ 0.10995498, -0.26081728,  0.95910596],
        [ 0.69665927, -0.66803262, -0.26153065]]))